In [1]:
from transformers.generation_utils import *

In [2]:
import os
import sys
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import transformers
import datasets

print(f"Running on transformers v{transformers.__version__} and datasets v{datasets.__version__}")

Running on transformers v4.9.1 and datasets v1.10.2


In [4]:
sys.path.append('../src-py')

import transformers
import datasets
from mt_bart_v2 import *

print(f"Running on transformers v{transformers.__version__} and datasets v{datasets.__version__}")

Running on transformers v4.9.1 and datasets v1.10.2


In [5]:
tokenizer = BartTokenizer.from_pretrained('../data/output/joint-con-counter-bart-model-no-attention/')

#joint_attention_model = BartModelV2.from_pretrained('../data/output/joint-con-counter-bart-model-with-joint-attention/', conc_loss_weight = 0.2, counter_loss_weight=0.8, attention_to_conc=True, conc_decoder=True).to(device)
nojoint_attention_model     = BartModelV2.from_pretrained('../data/output/joint-con-counter-bart-model-no-attention-finetuned/0-1-1-0/', conc_loss_weight = 0.1, counter_loss_weight=1.0, attention_to_conc=False, conc_decoder=True).to(device)
#noconc_decoder_model = BartModelV2.from_pretrained('../data/output/joint-con-counter-bart-model-no-conc-decoder/', conc_loss_weight = 0.2, counter_loss_weight=0.8, attention_to_conc=False, conc_decoder=False).to(device)

# tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
# model     = BartModelV2.from_pretrained('facebook/bart-base', conc_loss_weight = 0.5, counter_loss_weight=0.5).to(device)
# original_bart_model = BartModel.from_pretrained('facebook/bart-base').to(device)
# #load the weights of the two decoders
# model.conclusion_decoder.load_state_dict(original_bart_model.decoder.state_dict())
# model.counter_decoder.load_state_dict(original_bart_model.decoder.state_dict())

### Evaluating on Eval data:

In [6]:
gen_kwargs = {
    "do_sample": False, 
    "max_length":200,
    "top_p":0.95, 
    "top_k":50,
    "num_beams":1
}

In [7]:
ceph_dir = '/home/sile2804/data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation'
local_home_dir = '../data'

valid_df = pd.read_pickle(ceph_dir + '/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')

In [8]:
valid_sample = valid_df.sample(5)

In [9]:
premises = [' '.join(x) for x in valid_sample.masked_premises.tolist()]
premise_input_ids = [tokenizer([x], return_tensors="pt").input_ids.to(device) for x in premises]

In [10]:
#valid_sample['noconc_counters']  = [tokenizer.decode(noconc_decoder_model.generate(x, **gen_kwargs)[1][0], skip_special_tokens=True) for x in premise_input_ids]
valid_sample['con_noattention_counters']  = [tokenizer.decode(nojoint_attention_model.generate(x, **gen_kwargs)[0][0], skip_special_tokens=True) for x in premise_input_ids]
#valid_sample['full_model_counters']  = [tokenizer.decode(joint_attention_model.generate(x, **gen_kwargs)[1][0], skip_special_tokens=True) for x in premise_input_ids]

In [11]:
valid_sample[['masked_premises', 'title', 'con_noattention_counters']].head()

,masked_premises,title,con_noattention_counters
323018,"[i see appeals to sympathy all the time using the lives of children., somehow, something is worse or more worthy of caring if it primarily affects the lives of children., i see no reason to value the lives of children over an adult., logically i think it makes sense to value adults over children ., adults are much more likely to have accomplished something in their lives, they're more likely to be skilled in a particular field and more likely to be able to benefit society right now ., they have had more time to build meaningful relationships with a larger number of people, and their relationships will be much more than just 'they're a child family, so i automatically care about them' i know this is an extremely unpopular opinion., i get extremely strange looks irl anytime i even suggest that i hold this view., so cmv!]",I don't value the lives of children more than the lives of adults.,"i think the problem is that the people who are doing the things that are worth the money are the people that are doing them. the people in the world are the ones who are making the money. the ones that are going to make the money, the people with the money and the people around them are the only people who will be able to pay for the money they are doing. the only thing that is going to help the people of the world is the people they are making money."
301385,"[the second amendment to the constitution is the most cited defense of gun ownership in the usa., the claim is that gun ownership is allowed, largely unrestricted, because the constitution assures all law abiding citizens this right., ccw holders, specifically, are claimed to have the one of the lowest crime rates in american society, a question that i have had recently is this: what if the constitution was changed to carve out more restricted access to guns?, would the class of 'law abiding' ccw holders and gun owners comply with the new laws?, or would they see this change as unconstitutional and disobey the new law?, this act, of course, would render them no longer 'law abiding. ', i believe that no, most previously law abiding gun owners would not give up their weapons, despite now being criminals., if the argument is that gun owners are the most law abiding citizens, it follows that they should obey the new law., this is regardless of the morality, effectiveness, or popularity of the law law abiders follow all laws as closely as possible., many law abiding gun owners claim to respect the constitution, however, i honestly believe that if the second amendment were changed that most previously law abiding gun owners would not respect the new law., this does not make sense if the claim is that gun owners respect the constitution, since once an amendment is in the constitution it quite literally becomes constitutional., this would mean that law abiding gun owners do not actually care about following the law or holding up the constitution., what they actually care about is access to guns, and possibly upholding certain family or regional traditions., therefore gun owners do not actually respect the constitution, they just agree with the constitution because it justifies their beliefs in gun ownership., so can you change my view reddit?, my view is that 'law abiding' gun owners do not believe in the rule of law to the extent that they claim., if the constitution were changed to place more restrictions on access to firearms, most previously law abiding gun owners would rather become criminals than follow the new laws., therefore gun ownership has nothing to do with respect for the constitution or law and only has to do with the preservation of american gun culture.]",I don't think that law-abiding gun owners actually respect the constitution the way that they claim to,"i think the problem with your view is that you are not saying that the constitution is the most effective way to make the law. you are saying that you don't care about th